<a href="https://colab.research.google.com/github/RiturajSingh2004/Week1/blob/main/Week1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

class CropAndFertilizerRecommender:
    def __init__(self):
        # Load and prepare data
        self.load_and_prepare_data()

        # Train models
        self.train_models()

    def load_and_prepare_data(self):
        # Load datasets
        fertilizer_df = pd.read_csv('Fertilizer Prediction.csv')
        crop_df = pd.read_csv('Crop_recommendation.csv')

        # Prepare fertilizer data
        self.fertilizer_data = pd.get_dummies(fertilizer_df,
                                              columns=['Soil Type', 'Crop Type'])
        self.fertilizer_X = self.fertilizer_data.drop('Fertilizer Name', axis=1)
        self.fertilizer_y = self.fertilizer_data['Fertilizer Name']

        # Prepare crop data
        self.crop_X = crop_df.drop('label', axis=1)
        self.crop_y = crop_df['label']

    def train_models(self):
        # Fertilizer Model
        X_train_fert, X_test_fert, y_train_fert, y_test_fert = train_test_split(
            self.fertilizer_X, self.fertilizer_y, test_size=0.2, random_state=42
        )

        scaler_fert = StandardScaler()
        X_train_fert_scaled = scaler_fert.fit_transform(X_train_fert)
        X_test_fert_scaled = scaler_fert.transform(X_test_fert)

        self.fertilizer_model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.fertilizer_model.fit(X_train_fert_scaled, y_train_fert)

        # Crop Model
        X_train_crop, X_test_crop, y_train_crop, y_test_crop = train_test_split(
            self.crop_X, self.crop_y, test_size=0.2, random_state=42
        )

        scaler_crop = StandardScaler()
        X_train_crop_scaled = scaler_crop.fit_transform(X_train_crop)
        X_test_crop_scaled = scaler_crop.transform(X_test_crop)

        self.crop_model = RandomForestClassifier(n_estimators=100, random_state=42)
        self.crop_model.fit(X_train_crop_scaled, y_train_crop)

        # Evaluate and visualize models
        self.visualize_model_performance(
            self.fertilizer_model, X_test_fert_scaled, y_test_fert,
            "Fertilizer Recommendation Model", "fertilizer_confusion_matrix.png"
        )
        self.visualize_model_performance(
            self.crop_model, X_test_crop_scaled, y_test_crop,
            "Crop Recommendation Model", "crop_confusion_matrix.png"
        )

    def visualize_model_performance(self, model, X_test, y_test, title, filename):
        # Predict and calculate metrics
        y_pred = model.predict(X_test)

        # Create confusion matrix
        plt.figure(figsize=(10, 8))
        cm = confusion_matrix(y_test, y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'{title} Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig(filename)
        plt.close()

        # Print detailed classification report
        print(f"\n{title} Performance:")
        print(classification_report(y_test, y_pred))

    def detailed_recommendation(self, recommendation_type, **kwargs):
        """
        Provide a comprehensive recommendation with explanations
        """
        if recommendation_type == 'fertilizer':
            # Fertilizer recommendation logic
            fertilizer = self.recommend_fertilizer(**kwargs)

            # Create a detailed report
            report = f"""
🌱 FERTILIZER RECOMMENDATION REPORT 🌱

Input Parameters:
- Temperature: {kwargs['temperature']}°C
- Humidity: {kwargs['humidity']}%
- Moisture: {kwargs['moisture']}%
- Soil Type: {kwargs['soil_type']}
- Crop Type: {kwargs['crop_type']}
- Nitrogen Level: {kwargs['nitrogen']}
- Potassium Level: {kwargs['potassium']}
- Phosphorous Level: {kwargs['phosphorous']}

🔬 Recommended Fertilizer: {fertilizer}

Recommendation Insights:
- This fertilizer is optimized for the given crop and soil conditions
- The recommendation considers nutrient balance and crop requirements
- Suggested application will help maximize crop yield and soil health
"""
            return report

        elif recommendation_type == 'crop':
            # Crop recommendation logic
            crop = self.recommend_crop(**kwargs)

            # Create a detailed report
            report = f"""
🌾 CROP RECOMMENDATION REPORT 🌾

Input Soil & Climate Parameters:
- Nitrogen: {kwargs['nitrogen']}
- Phosphorous: {kwargs['phosphorous']}
- Potassium: {kwargs['potassium']}
- Temperature: {kwargs['temperature']}°C
- Humidity: {kwargs['humidity']}%
- Soil pH: {kwargs['ph']}
- Rainfall: {kwargs['rainfall']} mm

🌿 Recommended Crop: {crop}

Recommendation Insights:
- This crop is best suited to your current soil and climate conditions
- Optimal for the nutrient profile and environmental parameters
- Highest predicted success based on machine learning analysis
"""
            return report

    def recommend_fertilizer(self, temperature, humidity, moisture,
                              soil_type, crop_type, nitrogen, potassium, phosphorous):
        # Existing recommendation logic
        input_data = pd.DataFrame({
            'Temparature': [temperature],
            'Humidity': [humidity],
            'Moisture': [moisture],
            'Soil Type': [soil_type],
            'Crop Type': [crop_type],
            'Nitrogen': [nitrogen],
            'Potassium': [potassium],
            'Phosphorous': [phosphorous]
        })

        input_encoded = pd.get_dummies(input_data)

        for col in self.fertilizer_X.columns:
            if col not in input_encoded.columns:
                input_encoded[col] = 0

        input_encoded = input_encoded.reindex(columns=self.fertilizer_X.columns, fill_value=0)

        scaler = StandardScaler()
        scaler.fit(self.fertilizer_X)
        input_scaled = scaler.transform(input_encoded)

        prediction = self.fertilizer_model.predict(input_scaled)
        return prediction[0]

    def recommend_crop(self, nitrogen, phosphorous, potassium,
                        temperature, humidity, ph, rainfall):
        # Existing recommendation logic
        input_data = np.array([[nitrogen, phosphorous, potassium,
                                temperature, humidity, ph, rainfall]])

        scaler = StandardScaler()
        scaler.fit(self.crop_X)
        input_scaled = scaler.transform(input_data)

        prediction = self.crop_model.predict(input_scaled)
        return prediction[0]

# Example usage
recommender = CropAndFertilizerRecommender()

# Detailed Fertilizer Recommendation
print(recommender.detailed_recommendation(
    'fertilizer',
    temperature=30, humidity=60, moisture=40,
    soil_type='Loamy', crop_type='Wheat',
    nitrogen=20, potassium=10, phosphorous=30
))

# Detailed Crop Recommendation
print(recommender.detailed_recommendation(
    'crop',
    nitrogen=90, phosphorous=40, potassium=45,
    temperature=21, humidity=82, ph=6.5, rainfall=203
))


Fertilizer Recommendation Model Performance:
              precision    recall  f1-score   support

    10-26-26       1.00      0.50      0.67         2
    14-35-14       1.00      1.00      1.00         1
    17-17-17       0.00      0.00      0.00         0
       20-20       1.00      1.00      1.00         1
       28-28       1.00      1.00      1.00         5
         DAP       1.00      1.00      1.00         5
        Urea       1.00      1.00      1.00         6

    accuracy                           0.95        20
   macro avg       0.86      0.79      0.81        20
weighted avg       1.00      0.95      0.97        20



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Crop Recommendation Model Performance:
              precision    recall  f1-score   support

       apple       1.00      1.00      1.00        23
      banana       1.00      1.00      1.00        21
   blackgram       1.00      1.00      1.00        20
    chickpea       1.00      1.00      1.00        26
     coconut       1.00      1.00      1.00        27
      coffee       1.00      1.00      1.00        17
      cotton       1.00      1.00      1.00        17
      grapes       1.00      1.00      1.00        14
        jute       0.92      1.00      0.96        23
 kidneybeans       1.00      1.00      1.00        20
      lentil       0.92      1.00      0.96        11
       maize       1.00      1.00      1.00        21
       mango       1.00      1.00      1.00        19
   mothbeans       1.00      0.96      0.98        24
    mungbean       1.00      1.00      1.00        19
   muskmelon       1.00      1.00      1.00        17
      orange       1.00      1.00      1.

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
